In [ ]:
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import  HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from htmlTemplates import css, bot_template, user_template
from langchain.llms import HuggingFaceHub

In [3]:
%pip install -r requirements.txt

     ---------------------------------------- 0.0/57.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.4 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.4 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.4 kB ? eta -:--:--
     -------------------- ----------------- 30.7/57.4 kB 262.6 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.4 kB 262.6 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.4 kB 217.9 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.4 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 57.4/57.4 kB 251.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Gettin

  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      building 'faiss._swigfaiss' extension
      swigging faiss\faiss\python\swigfaiss.i to faiss\faiss\python\swigfaiss_wrap.cpp
      swig.exe -python -c++ -Doverride= -I/usr/local/include -Ifaiss -doxygen -DSWIGWIN -module swigfaiss -o faiss\faiss\python\swigfaiss_wrap.cpp faiss\faiss\python\swigfaiss.i
      error: command 'swig.exe' failed: None
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for faiss-cpu
  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [37 lines of output]
      running bdist_wheel
      running build
      running build_py
      c

   --------------------------------- ------ 8.0/9.6 MB 493.9 kB/s eta 0:00:04
   --------------------------------- ------ 8.0/9.6 MB 493.7 kB/s eta 0:00:04
   --------------------------------- ------ 8.0/9.6 MB 492.9 kB/s eta 0:00:04
   --------------------------------- ------ 8.0/9.6 MB 493.3 kB/s eta 0:00:04
   --------------------------------- ------ 8.0/9.6 MB 493.3 kB/s eta 0:00:04
   --------------------------------- ------ 8.0/9.6 MB 492.0 kB/s eta 0:00:04
   --------------------------------- ------ 8.0/9.6 MB 491.9 kB/s eta 0:00:04
   --------------------------------- ------ 8.1/9.6 MB 492.1 kB/s eta 0:00:04
   --------------------------------- ------ 8.1/9.6 MB 491.8 kB/s eta 0:00:04
   --------------------------------- ------ 8.1/9.6 MB 492.2 kB/s eta 0:00:04
   --------------------------------- ------ 8.1/9.6 MB 492.2 kB/s eta 0:00:04
   --------------------------------- ------ 8.1/9.6 MB 490.9 kB/s eta 0:00:04
   --------------------------------- ------ 8.1/9.6 MB 490.8 kB/

In [4]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


In [5]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


In [6]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore



In [7]:
def get_conversation_chain(vectorstore):
    llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain


In [8]:
def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)


In [15]:
from dotenv import load_dotenv
def main():
    load_dotenv()
    st.set_page_config(page_title="Chat with multiple PDFs",
                       page_icon=":books:")
    st.write(css, unsafe_allow_html=True)

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    st.header("Chat with multiple PDFs :books:")
    user_question = st.text_input("Ask a question about your documents:")
    if user_question:
        handle_userinput(user_question)

    with st.sidebar:
        st.subheader("Your documents")
        pdf_docs = st.file_uploader(
            "Add your Test data here and click on 'Process'", accept_multiple_files=True)
        if st.button("Process"):
            with st.spinner("Processing"):
                # get pdf text
                raw_text = get_pdf_text(pdf_docs)

                # get the text chunks
                text_chunks = get_text_chunks(raw_text)

                # create vector store
                vectorstore = get_vectorstore(text_chunks)

                # create conversation chain
                st.session_state.conversation = get_conversation_chain(
                    vectorstore)



In [16]:

if __name__ == '__main__':
    main()

NameError: name 'st' is not defined

In [ ]:
!streamlit run C:\Users\danik\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]

In [14]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
